In [ ]:
import pytz
import os, sys, csv
import django
from dateutil import parser
from django.conf import settings
from django.utils.timezone import make_aware
from django.db.models.fields import DateTimeField, DateField, BooleanField, PositiveIntegerField, IntegerField
from esr21_subject.models import InformedConsent, EligibilityConfirmation, VaccinationHistory, OnSchedule, SubjectVisit
from edc_registration.models import RegisteredSubject
from edc_visit_schedule.models import SubjectScheduleHistory
records = []
count = 0
with open('/home/esr21/data_restore_202208212126/subjectschedulehistory.csv', newline='') as file:
# with open('/Users/ckgathi/Downloads/data_restore_202208212126/subjectschedulehistory.csv', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row = dict(row)
        model_fields = SubjectScheduleHistory._meta.fields
        for field in model_fields:
            field_name = field.name
            if getattr(field, 'choices'):
                choices = getattr(field, 'choices')
                field_value = [choice[0] for choice in choices if choice[1] == row.get(field_name)]
                if field_value:
                    row.update({f'{field_name}': field_value[0]})
                else:
                    row.update({f'{field_name}': None})
            elif isinstance(field, DateTimeField):
                try:
                    field_value = parser.parse(row.get(field_name))
                    field_value = field_value.astimezone(pytz.timezone(settings.TIME_ZONE))
                except parser.ParserError:
                    row.update({f'{field_name}': None})
                else:
                    row.update({f'{field_name}': field_value})
            elif isinstance(field, DateField):
                try:
                    field_value = parser.parse(row.get(field_name) or '').date()
                except parser.ParserError:
                    row.update({f'{field_name}': None})
                else:
                    row.update({f'{field_name}': field_value})
            elif isinstance(field, BooleanField):
                field_value = True if row.get(field_name) == 'True' else False
                row.update({f'{field_name}': field_value})
            elif isinstance(field, (PositiveIntegerField, IntegerField, )):
                try:
                    field_value = int(row.get(field_name) or '')
                except ValueError:
                    row.update({f'{field_name}': None})
                else:
                    row.update({f'{field_name}': field_value})
            elif field_name == 'site':
                site_name = row.get('site')
                try:
                    site = Site.objects.get(domain=site_name)
                except Site.DoesNotExist:
                    raise ValidationError('Site Does not exist')
                else:
                    row.update({f'{field_name}': site})
        records.append(row)

for record in records:
    del record['']
    obj = SubjectScheduleHistory(**record)
    obj.save_base(raw=True)